# Streaming

Sensors, IoT devices, social networks, and online transactions all generate data that needs to be monitored constantly and acted upon quickly. As a result, the need for large-scale, real-time stream processing is more evident than ever before. This tutorial module introduces Structured Streaming, the main model for handling streaming datasets in Apache Spark. *In Structured Streaming, a data stream is treated as a table that is being continuously appended*. This leads to a stream processing model that is very similar to a batch processing model. You express your streaming computation as a standard batch-like query as on a static table, but Spark runs it as an incremental query on the unbounded input table.


## Load data

Databricks has sample event data as files in`/databricks-datasets/structured-streaming/events/` to use to build a Structured Streaming application. Let's take a look at the contents of this directory.

This would work in databricks: 

    %fs ls /databricks-datasets/structured-streaming/events/

In [2]:
!ls datasets/events

file-0.json   file-18.json  file-27.json  file-36.json	file-45.json
file-1.json   file-19.json  file-28.json  file-37.json	file-46.json
file-10.json  file-2.json   file-29.json  file-38.json	file-47.json
file-11.json  file-20.json  file-3.json   file-39.json	file-48.json
file-12.json  file-21.json  file-30.json  file-4.json	file-49.json
file-13.json  file-22.json  file-31.json  file-40.json	file-5.json
file-14.json  file-23.json  file-32.json  file-41.json	file-6.json
file-15.json  file-24.json  file-33.json  file-42.json	file-7.json
file-16.json  file-25.json  file-34.json  file-43.json	file-8.json
file-17.json  file-26.json  file-35.json  file-44.json	file-9.json


Each line in the file contains a JSON record with two fields: `time` and `action`.

```
{"time":1469501675,"action":"Open"}
{"time":1469501678,"action":"Close"}{"time":1469501680,"action":"Open"}{"time":1469501685,"action":"Open"}{"time":1469501686,"action":"Open"}{"time":1469501689,"action":"Open"}{"time":1469501691,"action":"Open"}{"time":1469501694,"action":"Open"}{"time":1469501696,"action":"Close"}{"time":1469501702,"action":"Open"}{"time":1469501703,"action":"Open"}{"time":1469501704,"action":"Open"}
```

## Initialize the stream

Since the sample data is just a static set of files, you can emulate a stream from them by reading one file at a time, in the chronological order in which they were created:

```python
streamingInputDF = (
  spark.readStream\
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)
```

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MyApp").getOrCreate()

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.sql.functions import window

In [5]:
#inputPath = "/databricks-datasets/structured-streaming/events/" # databricks
inputPath = "./datasets/events/"

# Define the schema to speed up processing
jsonSchema = StructType([ StructField("time", TimestampType(), True), StructField("action", StringType(), True) ])

streamingInputDF = (
  spark.readStream\
    .schema(jsonSchema)\
    .option("maxFilesPerTrigger", 1)\
    .json(inputPath)
)

streamingCountsDF = (
  streamingInputDF
    .groupBy(
      streamingInputDF.action,
      window(streamingInputDF.time, "1 hour"))
    .count()
)

## Start the streaming job

You start a streaming computation by defining a sink and starting it. In our case, to query the counts interactively, set the complete set of 1 hour counts to be in an in-memory table. The command below essentially simulates a time series generator in real time, using the files in the directory.

In [6]:
query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only)
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

`query` is a handle to the streaming query named `counts` that is running in the background. This query continuously picks up files and updates the windowed counts. The command window reports the status of the stream.

## Query the stream in real time

We now periodically query the `counts` aggregation:

In [8]:
result=spark.sql("select action, date_format(window.end, 'MMM-dd HH:mm') as time, count from counts order by time, action")

In [18]:
result.show()

+------+------------+-----+
|action|        time|count|
+------+------------+-----+
| Close|Jul-26 03:00|   11|
|  Open|Jul-26 03:00|  179|
| Close|Jul-26 04:00|  344|
|  Open|Jul-26 04:00| 1001|
| Close|Jul-26 05:00|  815|
|  Open|Jul-26 05:00|  999|
| Close|Jul-26 06:00| 1003|
|  Open|Jul-26 06:00| 1000|
| Close|Jul-26 07:00| 1011|
|  Open|Jul-26 07:00|  993|
| Close|Jul-26 08:00|  989|
|  Open|Jul-26 08:00| 1008|
| Close|Jul-26 09:00|  318|
|  Open|Jul-26 09:00|  329|
| Close|Jul-26 14:00|  699|
|  Open|Jul-26 14:00|  656|
| Close|Jul-26 15:00|  994|
|  Open|Jul-26 15:00|  991|
| Close|Jul-26 16:00|  988|
|  Open|Jul-26 16:00| 1020|
+------+------------+-----+
only showing top 20 rows



The query would change every time you execute it to reflect the action count based on the input stream of data.